In [ ]:
!pip install pyspark
!pip install --upgrade google-cloud-bigtable

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Web Content Indexing").getOrCreate()
json_df = spark.read.json("part-r-00000")
json_df.show()

+--------------------+--------------------+--------------------+-----------------+-----------------+--------------------+------------+--------------------+----------+-------------+--------------------+
|           fetchTime|            metadata|        modifiedTime|retriesSinceFetch|retryIntervalDays|retryIntervalSeconds|       score|           signature|statusCode|   statusName|                 url|
+--------------------+--------------------+--------------------+-----------------+-----------------+--------------------+------------+--------------------+----------+-------------+--------------------+
|Mon Nov 13 21:20:...|{NULL, NULL, NULL...|Thu Jan 01 07:30:...|                0|               30|             2592000|4.0064103E-4|                null|         1| db_unfetched|  http://www.3u.com/|
|Wed Dec 13 21:00:...|{text/html, temp_...|Mon Nov 13 21:00:...|                0|               30|             2592000| 0.106155306|                null|         4|db_redir_temp|   https://a

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window

json_df = json_df.dropDuplicates().na.drop()

tokenizer = Tokenizer(inputCol="url", outputCol="keywords")
json_df = tokenizer.transform(json_df)

windowSpec = Window.orderBy(col("score").desc())
ranked_json_df = json_df.withColumn("rank", rank().over(windowSpec))

indexed_json_df = ranked_json_df.select("url", "keywords", "rank")
indexed_json_df.write.parquet("output")

In [2]:
parquet_df = spark.read.parquet("output/part-00000-7170e01f-c3fb-47a4-b6c9-9e7cce45d50a-c000.snappy.parquet")
parquet_df.show()

+--------------------+--------------------+----+
|                 url|            keywords|rank|
+--------------------+--------------------+----+
|   https://amanz.my/| [https://amanz.my/]|   1|
|https://techlagi....|[https://techlagi...|   2|
|https://fonts.goo...|[https://fonts.go...|   3|
|https://techlagi....|[https://techlagi...|   4|
|   https://amanz.me/| [https://amanz.me/]|   5|
|https://www.googl...|[https://www.goog...|   6|
|https://techlagi....|[https://techlagi...|   7|
|https://techlagi....|[https://techlagi...|   8|
|https://techlagi.my/|[https://techlagi...|   9|
|https://techlagi....|[https://techlagi...|  10|
|https://techlagi....|[https://techlagi...|  11|
|https://amanz.my/...|[https://amanz.my...|  12|
|https://stats.wp....|[https://stats.wp...|  12|
|https://amanz.my/...|[https://amanz.my...|  12|
|https://amanz.my/...|[https://amanz.my...|  12|
|https://fonts.gst...|[https://fonts.gs...|  12|
|https://static.cl...|[https://static.c...|  12|
|https://amanz.my/..

In [3]:
import os
from google.cloud import bigtable
from google.oauth2 import service_account

project_id = "web-content-indexing"
instance_id = "data-engineering-project"
table_id = "crawled-links"
parquet_file = "output/part-00000-7170e01f-c3fb-47a4-b6c9-9e7cce45d50a-c000.snappy.parquet"

# Set the path to your service account JSON key file
credentials_path = "web-content-indexing-78a6a13bc8de.json"

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

# Create a Cloud Bigtable client using the service account credentials
client = bigtable.Client(project=project_id,
                         credentials=service_account.Credentials.from_service_account_file(credentials_path))

# Connect to an existing Cloud Bigtable instance
instance = client.instance(instance_id)

# Open an existing table or create a new one
table = instance.table(table_id)

# Read the Parquet file into a Spark DataFrame
parquet_df = spark.read.parquet(parquet_file)

# Convert the DataFrame to a Pandas DataFrame for easy iteration
pandas_df = parquet_df.toPandas()

# Iterate over rows and write to Bigtable
for index, row in pandas_df.iterrows():
    row_key = str(row['url'])
    column_family_id = "cf1"
    column_id = "c1"

    # Extract 'keywords' column value from the Pandas DataFrame
    keywords_value = str(row['keywords'])

    # Create a new row
    row_key_bytes = row_key.encode("utf-8")
    row = table.row(row_key_bytes)

    # Add the column value to the row
    row.set_cell(column_family_id, column_id, keywords_value.encode("utf-8"))  # Encode keywords_value to bytes

    # Apply the mutation to the table
    table.mutate_rows([row])

    print(f"Row with key '{row_key}' written to Bigtable.")

Row with key 'https://amanz.my/' written to Bigtable.
Row with key 'https://techlagi.my/berita/' written to Bigtable.
Row with key 'https://fonts.googleapis.com/' written to Bigtable.
Row with key 'https://techlagi.my/author/zafrul-aiman/' written to Bigtable.
Row with key 'https://amanz.me/' written to Bigtable.
Row with key 'https://www.googletagmanager.com/' written to Bigtable.
Row with key 'https://techlagi.my/ulasan-xiaomi-smart-air-purifier-4-kenapa-ia-berbaloi-untuk-dimiliki/' written to Bigtable.
Row with key 'https://techlagi.my/category/berita/' written to Bigtable.
Row with key 'https://techlagi.my/' written to Bigtable.
Row with key 'https://techlagi.my/category/berita/ulasan/' written to Bigtable.
Row with key 'https://techlagi.my/vroom%E2%9A%A1/' written to Bigtable.
Row with key 'https://amanz.my/xmlrpc.php' written to Bigtable.
Row with key 'https://stats.wp.com/' written to Bigtable.
Row with key 'https://amanz.my/comments/feed/' written to Bigtable.
Row with key 'htt

Row with key 'https://techlagi.my/teknologi-tokyo-2020/' written to Bigtable.
Row with key 'https://techlagi.my/kembara-5g-tech-lagi-di-malaysia-berdesup-perginya/' written to Bigtable.
Row with key 'https://techlagi.my/hyundai-beri-nafas-baru-kepada-grandeur-klasik-di-luar-tapi-canggih-di-dalam/' written to Bigtable.
Row with key 'https://techlagi.my/ulasan-aopen-27hc5rp-monitor-gaming-murah-dan-padu/' written to Bigtable.
Row with key 'https://techlagi.my/poco-x4-pro-5g-kini-rasmi-di-malaysia-apa-yang-perlu-anda-tahu/' written to Bigtable.
Row with key 'https://techlagi.my/acer-umumkan-siri-laptop-swift-baharu-lebih-berkuasa-premium-dan-ringan/' written to Bigtable.
Row with key 'https://techlagi.my/ulasan-xiaomi-mi-airdots-fon-telinga-nirwayar-yang-popular-dan-mampu-milik/' written to Bigtable.
Row with key 'https://techlagi.my/category/berita/lain-lain/' written to Bigtable.
Row with key 'https://techlagi.my/sony-xperia-1-iii-peranti-premium-di-jalan-yang-sepi/' written to Bigtable

Row with key 'https://techlagi.my/tag/iphone-se-2022/' written to Bigtable.
Row with key 'https://techlagi.my/tag/apple-event/' written to Bigtable.
Row with key 'https://techlagi.my/tag/iphone-se/' written to Bigtable.
Row with key 'https://techlagi.my/ulasan-acerpure-cool-kipas-2-dalam-1-yang-membersih-dan-menyejuk-udara-di-rumah/feed/' written to Bigtable.
Row with key 'https://twitter.com/Tenaga_Nasional' written to Bigtable.
Row with key 'https://www.facebook.com/TNBCareline' written to Bigtable.
Row with key 'https://techlagi.my/wp-json/wp/v2/posts/2280' written to Bigtable.
Row with key 'https://techlagi.my/wp-json/wp/v2/pages/879' written to Bigtable.
Row with key 'https://techlagi.my/tag/acerpure-cool/' written to Bigtable.
Row with key 'https://techlagi.my/cara-claim-barang-elektrik-rosak-akibat-litar-pintas-dari-tnb/feed/' written to Bigtable.
Row with key 'https://techlagi.my/wp-json/wp/v2/posts/2252' written to Bigtable.
Row with key 'https://techlagi.my/tag/voltan-tinggi/

In [ ]:
from google.cloud import bigtable
from google.oauth2 import service_account

project_id = "web-content-indexing"
instance_id = "data-engineering-project"
table_id = "crawled-links"

# Set the path to your service account JSON key file
credentials_path = "web-content-indexing-78a6a13bc8de.json"

# Create a Cloud Bigtable client using the service account credentials
client = bigtable.Client(project=project_id,
                         credentials=service_account.Credentials.from_service_account_file(credentials_path))

# Connect to an existing Cloud Bigtable instance
instance = client.instance(instance_id)

# Open an existing table
table = instance.table(table_id)

# Read data from Bigtable
rows = table.read_rows()

# Print the data
for row in rows:
    # Extract row key
    row_key = row.row_key.decode("utf-8")

    # Extract column family and column values
    columns_data = {}
    for family, columns in row.cells.items():
        for column, cells in columns.items():
            value = cells[0].value.decode("utf-8")
            columns_data[f"{family}:{column}"] = value

    # Print row data
    print(f"Row key: {row_key}, Data: {columns_data}")


In [ ]:
spark.stop()